In [1]:
# TODO: Dirty hack to import from sibling dir. Put wahlrecht_polling_firms.py into the same folder as this file eventually.
import sys
import os
sys.path.append(os.path.abspath('../Backend'))
from wahlrecht_polling_firms import get_tables
from days_to_weeks import week
from pandas import DataFrame
import numpy as np
import pandas as pd
import datetime as dt

In [32]:
#TODO: add the datum for every week
def average(data, model, weightvector=None):
    '''
    averages over the polling data of all firms according to the data available for each week.
    
    data: polling data and the model that should be used('simple','weightparticipants'or
    'weightfirms'(needs a weightdictionary with a weight for every firm))
    return: dictionary of parties with the average results for every week
    '''
    week_ind={}
    n_weeks = 0
    for key in data:
        wk = week(data[key])
        week_ind[key]= wk
        n_weeks = np.maximum(n_weeks,np.max(wk))
    
    n_parties=7
    result=np.zeros((n_weeks,n_parties))
    total_part = np.zeros(n_weeks)
    parties=['CDU/CSU','SPD','GRÜNE','FDP','LINKE','AfD','Sonstige']
    
    
    if model == 'simple':
        for i in np.arange (n_weeks):
            n = 0
            for key in data:               
                if i in week_ind[key]:                    
                    current_ind = np.where(week_ind[key]==i)[0][0]
                    total_part[i] += data[key]['Befragte'][current_ind]
                    j = 0
                    for p in parties:
                        result[i,j] += data[key][p][current_ind]
                        j += 1
                    n += 1
            result[i,:] /= n
    
    if model == 'weightparticipants':
        for i in np.arange(n_weeks):
            n = 0
            for key in data:
                if i in week_ind[key]:
                    current_ind = np.where(week_ind[key]==i)[0][0]
                    total_part[i] += data[key]['Befragte'][current_ind]
                    n_part = data[key]['Befragte'][current_ind]
                    j = 0
                    for p in parties:
                        result[i,j] += data[key][p][current_ind]*n_part
                        j += 1
                    n += n_part  
            result[i,:] /= n      
            
    if model == 'weightfirms':
        for i in np.arange(n_weeks):
            n = 0
            for key in data:
                if i in week_ind[key]:
                    current_ind = np.where(week_ind[key]==i)[0][0]
                    total_part[i] += data[key]['Befragte'][current_ind]
                    j = 0
                    for p in parties:
                        result[i,j] += data[key][p][current_ind]*weightvector[key]
                        j += 1
                    n += weightvector[key]  
            result[i,:] /= n           
    
    res_dict = {}
    j = 0
    for p in parties:
        res_dict[p] = result[:,j]
        j += 1
    res = pd.DataFrame.from_dict(res_dict)
    res['Befragte'] = total_part
    return res

    

In [22]:
#testing
data = get_tables()

In [23]:
w = {'allensbach':0.2, 'emnid':0.1, 'forsa':0.1, 'politbarometer':0.1, 'gms':0.2, 'dimap':0.1, 'insa':0.1}

In [37]:
res = average(data,'weightfirms',w)
print(res['LINKE'][:200])

/home/eric/Predictor/models/days_to_weeks.py:30: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  X = -(X - next_sunday).astype('timedelta64[D]').reshape(-1,1)
/home/eric/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:63: RuntimeWarning: invalid value encountered in true_divide


0      11.000000
1       9.000000
2       9.000000
3       8.500000
4       8.400000
5       8.200000
6       8.750000
7       8.700000
8       8.916667
9       8.750000
10      8.125000
11      8.500000
12      8.100000
13      7.750000
14      7.833333
15      7.900000
16      7.666667
17      8.083333
18      8.250000
19      8.400000
20      8.875000
21      9.571429
22     10.333333
23      9.750000
24      9.750000
25      9.000000
26      9.900000
27     10.000000
28     10.100000
29     10.166667
         ...    
170     9.714286
171     9.625000
172     9.875000
173     9.333333
174     9.666667
175     9.833333
176     9.000000
177     8.785714
178     9.200000
179     9.875000
180     9.250000
181    10.000000
182          NaN
183     9.000000
184     9.416667
185     9.375000
186     9.500000
187     9.250000
188     9.166667
189     9.000000
190     9.000000
191     8.916667
192     9.166667
193     9.000000
194     9.000000
195     9.500000
196     9.000000
197     8.6666

In [38]:
res

,AfD,CDU/CSU,FDP,GRÜNE,LINKE,SPD,Sonstige,Befragte
0,9.000000,36.500000,9.000000,6.500000,11.000000,25.000000,3.000000,2040.0
1,8.000000,38.375000,8.500000,7.125000,9.000000,24.125000,4.875000,7127.0
2,8.000000,38.500000,8.500000,7.500000,9.000000,24.000000,4.500000,7439.0
3,8.000000,38.083333,9.000000,7.166667,8.500000,24.833333,4.416667,8209.0
4,7.800000,37.500000,8.900000,7.400000,8.400000,25.600000,4.400000,7704.0
5,8.200000,37.600000,8.000000,7.000000,8.200000,26.400000,4.600000,9317.0
6,8.750000,36.250000,7.000000,7.500000,8.750000,27.500000,4.250000,7408.0
7,8.800000,35.600000,6.800000,7.100000,8.700000,28.500000,4.500000,7476.0
8,8.333333,35.833333,6.083333,7.083333,8.916667,30.000000,3.750000,9217.0
9,9.250000,35.250000,5.875000,6.250000,8.750000,30.375000,4.250000,6391.0
